In [1]:
import ltn
import tensorflow as tf
import numpy as np
from collections import defaultdict

In [ ]:
tf.config.run_functions_eagerly(True)

In [3]:
import h5py
imdb = h5py.File('../data/mini_imdb_1024.h5')
sgdb = h5py.File('../data/mini_VG-SGG.h5')

import json
with open('../data/mini_VG-SGG-dicts.json') as f:
    sgdicts = json.load(f)

In [78]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Equiv = ltn.Wrapper_Connective(ltn.fuzzy_ops.Equiv(ltn.fuzzy_ops.And_Prod(),ltn.fuzzy_ops.Implies_Reichenbach()))
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=6),semantics="exists")

In [178]:
images = ltn.Variable('image', [i for i in range(len(imdb['images']))])
objects = ltn.Variable('object', [i for i in range(len(sgdicts['label_to_idx']))])
relationships = ltn.Variable('relationship', [i for i in range(len(sgdicts['predicate_to_idx']))])
subjects = ltn.Variable('subject', [i for i in range(len(sgdicts['label_to_idx']))])

In [251]:
in_image = defaultdict(set)
for image in images.tensor:
    image = int(image[0])
    if sgdb['img_to_first_box'][image] >= 0:
        boxes = [b for b in range(sgdb['img_to_first_box'][image], sgdb['img_to_last_box'][image]+1)]
        in_image[image] = set([sgdb['labels'][b][0] for b in boxes])

def object_in_image(args):
    args = tf.stack(args, axis=1)
    output = [0] * args.shape[0]
    print(args.numpy())
    for i, arg in enumerate(args):
        object, image = arg
        object, image = int(object), int(image)

        if object in in_image[image]:
            output[i] = 1

    return output


obj_in_img = ltn.Predicate(object_in_image)

In [256]:
related_in_image = [defaultdict(set) for _ in range(len(imdb['images']))]
for image in images.tensor:
    image = int(image[0])

    if sgdb['img_to_first_rel'][image] >= 0:
        relations = [i for i in range(sgdb['img_to_first_rel'][image], sgdb['img_to_last_rel'][image]+1)]
        for relation in relations:
            predicate = sgdb['predicates'][relation][0]
            related_objects = tuple([sgdb['labels'][o][0] for o in sgdb['relationships'][relation].tolist()])
            related_in_image[image][predicate].add(related_objects)

def relationship_in_image(args):
    output = []

    args = args.numpy()
    for relationship, subject, object, image in zip(*args):
        relationship = int(relationship)
        subject, object = int(subject), int(object)
        image = int(image)

        if (subject, object) in related_in_image[image][relationship]:
            output.append(1)
        else:
            output.append(0)

    return output

rel_in_img = ltn.Predicate(relationship_in_image)

In [201]:
class ModelConnected(tf.keras.Model):
    def __init__(self):
        super(ModelConnected, self).__init__()
        self.dense1 = tf.keras.layers.Dense(32, activation=tf.nn.elu)
        self.dense2 = tf.keras.layers.Dense(16, activation=tf.nn.elu)
        self.dense3 = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)

    def call(self, inputs):

        relation, subject, object = inputs 
        x = tf.concat([relation, subject, object], axis=1)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3(x)
    
Connected = ltn.Predicate(ModelConnected())

In [291]:
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

@tf.function
def axioms():
    axioms = [
        Forall(
            [images, relationships, subjects, objects],
            Implies(
                And(
                    obj_in_img([subjects, images]),
                    obj_in_img([objects, images]),
                    rel_in_img([relationships, subjects, objects, images])
                ),
                Connected([relationships, subjects, objects])
            )
        )
    ]

    kb = formula_aggregator(axioms)
    sat = kb.tensor
    return sat

axioms()

AttributeError: in user code:

    File "/tmp/ipykernel_15369/637298240.py", line 5, in axioms  *
        axioms = [
    File "/home/grav/.local/lib/python3.10/site-packages/ltn/core.py", line 189, in __call__  *
        expr = super().__call__(inputs, *args, **kwargs)
    File "/home/grav/.local/lib/python3.10/site-packages/ltn/core.py", line 159, in __call__  *
        t_outputs = self.model(as_tensors(flat_inputs), *args, **kwargs)
    File "/tmp/ipykernel_15369/3831044745.py", line 11, in object_in_image  *
        print(args.numpy())

    AttributeError: 'Tensor' object has no attribute 'numpy'


In [292]:
accuracy = 0
relations = len(sgdb['relationships'])
for image in related_in_image:
    for relation in image:
        for subject, object in image[relation]:
            sub = ltn.Constant(subject, trainable=False)
            obj = ltn.Constant(object, trainable=False)
            
            prediction = int(tf.argmax(Connected([relationships, sub, obj]).tensor).numpy())
            if prediction == relation:
                accuracy += 1

print(f'ACCURACY: {accuracy / relations}')

ACCURACY: 0.000988875154511743


In [219]:
test = obj_in_img([objects, images])
test.tensor.shape

TensorShape([150, 1000])

In [273]:
hat = ltn.Constant(1, trainable=False)
image1 = ltn.Constant(0, trainable=False)

test = obj_in_img([hat, image1])
test.tensor.numpy()

[[[1.]
  [0.]]]


0.0

In [154]:
test.take('image', 1)

ltn.Term(tensor=0.0, free_vars=[])

In [175]:
touching = ltn.Constant(0, trainable=False)

test = rel_in_img([relationships, subjects, hat, image1])
test.tensor.shape

TensorShape([50, 150])

In [288]:
test = Connected([relationships, hat, hat])
am = tf.argmax(test.tensor)
am.numpy()
test

ltn.Formula(tensor=[0.70203084 0.71996546 0.7314017  0.75179565 0.7779284  0.8053186
 0.8314769  0.8554006  0.8768152  0.8957781  0.9124077  0.9268533
 0.9392937  0.9499198  0.9589253  0.9665008  0.9728282  0.97807753
 0.98239017 0.9858999  0.98874307 0.9910366  0.9928798  0.99435633
 0.99553543 0.9964743  0.9972198  0.9978106  0.99827784 0.99864686
 0.99893785 0.99916697 0.99934727 0.9994889  0.99960005 0.99968725
 0.99975556 0.9998091  0.9998509  0.99988365 0.9999092  0.99992925
 0.9999448  0.999957   0.9999665  0.9999739  0.9999797  0.9999842
 0.9999877  0.9999904 ], free_vars=['relationship'])

In [222]:
Forall(
    [images, subjects],
    obj_in_img([subjects, images])
)

ltn.Formula(tensor=0.01987755298614502, free_vars=[])

In [224]:
a = tf.constant([1, 2, 3])
b = tf.constant([4, 5, 6])

In [233]:
tf.stack([a, b], axis=1)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 4],
       [2, 5],
       [3, 6]], dtype=int32)>

In [276]:
len(sgdb['relationships'])

4045